# Data Collection Proof-of-Concept
##  -- Mega-Cov Hydrator

The Google Colab notebook shows to to hydrate an Mega-cov archive.

You need to run this on Google Colab to get tweet metadata including its text.

The demo below hydrates 1 month of Mega-Cov data, filter it by Country == Canada, Australia, etc. & Language == English. 


# Initializaition

In [1]:
from IPython.display import clear_output
%pip install twarc
%pip install jsonlines
# %pip install wget
%pip install jsonlines
clear_output()

import os
from google.colab import drive 
import pandas as pd
from datetime import datetime
# import wget
import json
import jsonlines
import tqdm
from twarc import Twarc


# shared archive -> "local" archive



Follow these steps:

1. Go to your Googe Drive
2. Create a folder named `MegaCov` 
3. Locate a Mega-cov archive file from [here](https://github.com/UBC-NLP/megacov/tree/master/tweet_ids) 
4. Add it to the `MegaCov` folder you just created in your Google Drive.


Now run the cell below.

When prompted, grant acess.

In [2]:

drive.mount('/content/gdrive')
clear_output()
working_directory = 'My Drive/MegaCov'

wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)


current directory is : /content/gdrive/My Drive/MegaCov


# archive -> json

In [6]:
path = os.path.join(wd, "tweet_ids") 
if not os.path.exists('./tweet_ids'):
  try:
    os.makedirs(path)
    # os.chdir('tweet_ids')
  except:
    pass

In [8]:
os.chdir('tweet_ids')
# !pwd

In [47]:
!tar -xvf  'MagaCOV_Apr_2020.tar.gz' -C './tweet_ids'
#TODO unzip in batch *.tar.gz files

4_2020_1.json
4_2020_2.json


#  json  + filter -> tweet_ids

In [49]:
os.chdir('tweet_ids')


In [50]:
!pwd

/content/gdrive/My Drive/MegaCov/tweet_ids


In [37]:
!ls

# file = '5_2020_1.json'


5_2020_1.json  5_2020_2.json


In [33]:
# list of countries we research:
ctrs = ['Canada',
        'United States',
        'Mexico',
        'Austrialia',
        'New Zealand',
        'United Kingdom',
        'France']

In [53]:
directory =  os.getcwd()
twt_ids = []
cnt = 0

for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".json"):
            with jsonlines.open(file) as reader:
                for obj in reader:

                    cnt += 1
                    
                    #  filter by country and by Language:
                    if obj['LangID_tool'] == 'en':
                        if obj['country'] in ctrs: 
                            twt_ids.append(obj['tweet_id'])

print('\n done!')


 done!


In [55]:
len(twt_ids)

170891

# tweet_ids -> txt

In [57]:
#@title Enter ID output file {run: "auto"}
os.chdir(wd)
final_tweet_ids_filename = "ids_2020Apr.txt" #@param {type: "string"}
# The set of IDs is stored in this file.
with open(final_tweet_ids_filename, "w+") as f:
    for id in twt_ids:
        f.write('%s\n' % id)

# Hydrate 

In [58]:
#@title Insert API Keys here { run : "auto"}
from twarc import Twarc

# These keys are received after applying for a twitter developer account

consumer_key = "" #@param {type:"string"}
consumer_secret = "" #@param {type:"string"}
access_token = "" #@param {type:"string"}
access_token_secret = "" #@param {type:"string"}

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

In [59]:
#@title Set up Directory { run: "auto"}
input_filename_tweet_ids = "ids_2020Apr.txt" #@param {type: "string"}
output_filename = "hydrated_2020Apr.tsv" #@param {type: "string"}

In [60]:
# Stores hydrated tweets here as jsonl objects
# Contains one json object per line
output_json_filename = output_filename[:output_filename.index(".")] + ".jsonl"
ids = []
with open(final_tweet_ids_filename, "r") as ids_file:
    ids = ids_file.read().split()
hydrated_tweets = []
ids_to_hydrate = set(ids)

# Looks at the output file for already hydrated tweets
if os.path.isfile(output_json_filename):
    with jsonlines.open(output_json_filename, "r") as reader:
        for i in reader.iter(type=dict, skip_invalid=True):
            # These tweets have already been hydrated. So remove them from ids_to_hydrate
            hydrated_tweets.append(i)
            ids_to_hydrate.remove(i["id_str"])
print("Total IDs: " + str(len(ids)) + ", IDs to hydrate: " + str(len(ids_to_hydrate)))
print("Hydrated: " + str(len(hydrated_tweets)))

count = len(hydrated_tweets)
start_index = count # The index from where tweets haven't been saved to the output_json_file
# Stores hydrated tweets to output_json_file every num_save iterations.
num_save  = 1000

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate):
    hydrated_tweets.append(tweet)
    count += 1
    # If num_save iterations have passed,
    if (count % num_save) == 0:
        # Open the output file
        # NOTE: Even if the code stops during IO, only tweets from the current iteration are lost.
        # Older tweets are preserved as the file is written in append mode.
        with jsonlines.open(output_json_filename, "a") as writer:
            print("Started IO")
            # Now write the tweets from start_index. The other tweets don't have to be written
            # as they were already written in a previous iteration or run.
            for hydrated_tweet in hydrated_tweets[start_index:]:
                writer.write(hydrated_tweet)
            print("Finished IO")
        print("Saved " + str(count) + " hydrated tweets.")
        # Now, since everything has been written. Reset start_index
        start_index = count
# There might be tweets unwritten in the last iteration if the count is not a multiple of num_tweets.
# In that case, just write out the remainder of tweets.
if count != start_index:
    print("Here with start_index", start_index)
    with jsonlines.open(output_json_filename, "a") as writer:
        for hydrated_tweet in hydrated_tweets[start_index:]:
           writer.write(hydrated_tweet)  
print('\n done!') 

Total IDs: 170891, IDs to hydrate: 170891
Hydrated: 0
Started IO
Finished IO
Saved 1000 hydrated tweets.
Started IO
Finished IO
Saved 2000 hydrated tweets.
Started IO
Finished IO
Saved 3000 hydrated tweets.
Started IO
Finished IO
Saved 4000 hydrated tweets.
Started IO
Finished IO
Saved 5000 hydrated tweets.
Started IO
Finished IO
Saved 6000 hydrated tweets.
Started IO
Finished IO
Saved 7000 hydrated tweets.
Started IO
Finished IO
Saved 8000 hydrated tweets.
Started IO
Finished IO
Saved 9000 hydrated tweets.
Started IO
Finished IO
Saved 10000 hydrated tweets.
Started IO
Finished IO
Saved 11000 hydrated tweets.
Started IO
Finished IO
Saved 12000 hydrated tweets.
Started IO
Finished IO
Saved 13000 hydrated tweets.
Started IO
Finished IO
Saved 14000 hydrated tweets.
Started IO
Finished IO
Saved 15000 hydrated tweets.
Started IO
Finished IO
Saved 16000 hydrated tweets.
Started IO
Finished IO
Saved 17000 hydrated tweets.
Started IO
Finished IO
Saved 18000 hydrated tweets.
Started IO
Finished

Started IO
Finished IO
Saved 79000 hydrated tweets.
Started IO
Finished IO
Saved 80000 hydrated tweets.
Started IO
Finished IO
Saved 81000 hydrated tweets.
Started IO
Finished IO
Saved 82000 hydrated tweets.
Started IO
Finished IO
Saved 83000 hydrated tweets.
Started IO
Finished IO
Saved 84000 hydrated tweets.
Started IO
Finished IO
Saved 85000 hydrated tweets.
Started IO
Finished IO
Saved 86000 hydrated tweets.
Started IO
Finished IO
Saved 87000 hydrated tweets.
Started IO
Finished IO
Saved 88000 hydrated tweets.
Started IO
Finished IO
Saved 89000 hydrated tweets.
Started IO
Finished IO
Saved 90000 hydrated tweets.
Started IO
Finished IO
Saved 91000 hydrated tweets.
Started IO
Finished IO
Saved 92000 hydrated tweets.
Started IO
Finished IO
Saved 93000 hydrated tweets.
Started IO
Finished IO
Saved 94000 hydrated tweets.
Started IO
Finished IO
Saved 95000 hydrated tweets.
Started IO
Finished IO
Saved 96000 hydrated tweets.
Started IO
Finished IO
Saved 97000 hydrated tweets.
Started IO
F

In [61]:
print('next...')

next...


In [63]:
# Convert jsonl to tsv
import csv, jsonlines
output_json_filename = output_filename[:output_filename.index(".")] + ".jsonl"
# These are the column name that are selected to be stored in the tsv
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]
hydrated_tweets = []
# Reads the current tweets
with jsonlines.open(output_json_filename, "r") as reader:
    for i in reader.iter(type=dict, skip_invalid=True):
        hydrated_tweets.append(i)
# Writes them out
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset,delimiter='\t') # if not working, change filename to .csv in and remove ,delimiter='\t'
    d.writeheader()
    d.writerows(hydrated_tweets)

print('\n done!')


 done!


Now the tsv should be in MegaCov folder.